first we mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


and then downloading the training data

In [ ]:
!gdown 1pe4_HgXb9dctFGJXVNlyNcKSXZeht0lX

unziping the data

In [ ]:
!unzip /content/car_img-test.zip

importing the libraries that we need

In [ ]:
import cv2
import os
import xml.etree.ElementTree as ET
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image

converting XML files to a CSV(later i find out it was not nessory)

In [ ]:
def xml_to_csv(xml_dir, output_csv):
    data = []
    for xml_file in sorted(os.listdir(xml_dir)):  #sorting the files
        if xml_file.endswith('.xml'):
            tree = ET.parse(os.path.join(xml_dir, xml_file))
            root = tree.getroot()
            filename = xml_file.split('.')[0]  
            for obj in root.findall('object'):
                name = obj.find('name').text 
                if name == 'کل ناحیه پلاک': 
                    bbox = obj.find('bndbox')
                    xmin = float(bbox.find('xmin').text)
                    ymin = float(bbox.find('ymin').text)
                    xmax = float(bbox.find('xmax').text)
                    ymax = float(bbox.find('ymax').text)
                    data.append([filename, xmin, ymin, xmax, ymax]) 
    df = pd.DataFrame(data, columns=['Image Name', 'xmin', 'ymin', 'xmax', 'ymax'])
    df.to_csv(output_csv, index=False)

xml_dir = '/content/test'

output_csv = '/content/test_cars_dataset.csv'

xml_to_csv(xml_dir, output_csv)


In [ ]:
data = pd.read_csv('/content/test_cars_dataset.csv')

In [ ]:
data.head()

writing finction to get images size

In [ ]:

def get_image_dimensions(image_folder):
    dimensions = []  
    for image_name in sorted(os.listdir(image_folder)):  
        if image_name.endswith('.jpg'): 
            image_path = os.path.join(image_folder, image_name)
            image = cv2.imread(image_path) 
            if image is not None:
                height, width, _ = image.shape 
                dimensions.append([image_name, width, height])
            else:
                print(f"could not read the image {image_name}")
    return dimensions

image_folder = '/content/test'
dimensions = get_image_dimensions(image_folder)

df = pd.DataFrame(dimensions, columns=['Image Name', 'Width', 'Height'])

print(df)

df.to_csv('/content/test_cars_dimensions.csv', index=False)

In [ ]:
csv_path = '/content/test_cars_dimensions.csv' 
df = pd.read_csv(csv_path)

df['Image Name'] = df['Image Name'].str.replace('.jpg', '', regex=False)

print(df)

output_csv_path = '/content/test_cars_dimensions.csv'
df.to_csv(output_csv_path, index=False)

In [ ]:
df_dimensions = pd.read_csv('/content/test_cars_dimensions.csv')

df_plates = pd.read_csv('/content/test_cars_dataset.csv')

print(df_dimensions.head())
print(df_plates.head())

merge two csv files to extract txt files for yolo dataset

In [ ]:
merged_plates_dimension = pd.merge(df_plates, df_dimensions, on='Image Name', how='inner')

print(merged_plates_dimension)

In [ ]:

merged_plates_dimension['x_center'] = ((merged_plates_dimension['xmin'] + merged_plates_dimension['xmax']) / 2) / merged_plates_dimension['Width']
merged_plates_dimension['y_center'] = ((merged_plates_dimension['ymin'] + merged_plates_dimension['ymax']) / 2) / merged_plates_dimension['Height']
merged_plates_dimension['width'] = (merged_plates_dimension['xmax'] - merged_plates_dimension['xmin']) / merged_plates_dimension['Width']
merged_plates_dimension['height'] = (merged_plates_dimension['ymax'] - merged_plates_dimension['ymin']) / merged_plates_dimension['Height']

merged_plates_dimension['class_id'] = 0

yolo_df = merged_plates_dimension[['Image Name', 'class_id', 'x_center', 'y_center', 'width', 'height']]

print(yolo_df.head())

In [ ]:
os.makedirs('labels', exist_ok=True)

for image_name, group in yolo_df.groupby('Image Name'):
    txt_path = f'labels/{image_name}.txt'
    with open(txt_path, 'w') as f:
        for _, row in group.iterrows():
            f.write(f"{row['class_id']} {row['x_center']:.6f} {row['y_center']:.6f} {row['width']:.6f} {row['height']:.6f}\n")

In [ ]:

images_and_xmls_dir = '/content/test'
labels_dir = '/content/labels'

dataset_dir = '/content/dataset'
os.makedirs(os.path.join(dataset_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'labels'), exist_ok=True)

for file_name in os.listdir(images_and_xmls_dir):
    if file_name.endswith('.jpg'):
        src_image = os.path.join(images_and_xmls_dir, file_name)
        dst_image = os.path.join(dataset_dir, 'images', file_name)
        shutil.copy(src_image, dst_image)

for file_name in os.listdir(labels_dir):
    if file_name.endswith('.txt'):
        src_label = os.path.join(labels_dir, file_name)
        dst_label = os.path.join(dataset_dir, 'labels', file_name)
        shutil.copy(src_label, dst_label)

In [ ]:

labels_dir = '/content/dataset/'

drive_labels_dir = '/content/drive/MyDrive/dataset/'

shutil.copytree(labels_dir, drive_labels_dir)

In [ ]:

image_files = os.listdir('/content/dataset/images')
label_files = [f.replace('.jpg', '.txt') for f in image_files]

train_images, val_images, train_labels, val_labels = train_test_split(
    image_files, label_files, test_size=0.2, random_state=42
)

os.makedirs('/content/split_dataset/train/images', exist_ok=True)
os.makedirs('/content/split_dataset/train/labels', exist_ok=True)
os.makedirs('/content/split_dataset/val/images', exist_ok=True)
os.makedirs('/content/split_dataset/val/labels', exist_ok=True)

for img, lbl in zip(train_images, train_labels):
    shutil.copy(f'/content/dataset/images/{img}', f'/content/split_dataset/train/images/{img}')
    shutil.copy(f'/content/dataset/labels/{lbl}', f'/content/split_dataset/train/labels/{lbl}')

for img, lbl in zip(val_images, val_labels):
    shutil.copy(f'/content/dataset/images/{img}', f'/content/split_dataset/val/images/{img}')
    shutil.copy(f'/content/dataset/labels/{lbl}', f'/content/split_dataset/val/labels/{lbl}')

generate YML file for YOLO

In [ ]:
yaml_content = """
path: /content/split_dataset
train: /content/split_dataset/train/images
val: /content/split_dataset/val/images
nc: 1
names: ['license_plate']
"""

with open('/content/split_dataset/dataset.yaml', 'w') as f:
    f.write(yaml_content)

In [ ]:

labels_dir = '/content/split_dataset/'

drive_labels_dir = '/content/drive/MyDrive/split_dataset/'

shutil.copytree(labels_dir, drive_labels_dir)

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.9/921.9 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.80 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 35.5/112.6 GB disk)


In [ ]:
!yolo task=detect mode=train model=yolo11s.pt data='/content/drive/MyDrive/split_dataset/dataset.yaml' epochs=50 batch=32 imgsz=640 device=0 optimizer='AdamW' cos_lr=True lr0=0.001 lrf=0.1 plots=True

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/drive/MyDrive/split_dataset/dataset.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

In [ ]:
!zip -r runs.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/train2/ (stored 0%)
  adding: content/runs/detect/train2/weights/ (stored 0%)
  adding: content/runs/detect/train2/args.yaml (deflated 52%)
  adding: content/runs/detect/train3/ (stored 0%)
  adding: content/runs/detect/train3/train_batch4202.jpg (deflated 9%)
  adding: content/runs/detect/train3/val_batch2_labels.jpg (deflated 11%)
  adding: content/runs/detect/train3/F1_curve.png (deflated 17%)
  adding: content/runs/detect/train3/confusion_matrix_normalized.png (deflated 35%)
  adding: content/runs/detect/train3/val_batch1_labels.jpg (deflated 11%)
  adding: content/runs/detect/train3/events.out.tfevents.1740489344.cf42acd0e7f2.1916.0 (deflated 91%)
  adding: content/runs/detect/train3/weights/ (stored 0%)
  adding: content/runs/detect/train3/weights/last.pt (deflated 8%)
  adding: content/runs/detect/train3/weights/best.pt (deflated 8%)
  adding: content/runs/detect/train3/v

In [ ]:
import shutil
labels_dir = '/content/runs.zip'

drive_labels_dir = '/content/drive/MyDrive/runs_yolo11s'

shutil.copy(labels_dir, drive_labels_dir)

'/content/drive/MyDrive/runs_yolo11s'

In [ ]:
!gdown 1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT

Downloading...
From (original): https://drive.google.com/uc?id=1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT
From (redirected): https://drive.google.com/uc?id=1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT&confirm=t&uuid=fa961c57-15df-47fa-8250-ad1fabaf6db8
To: /content/car_img-validation.zip
100% 1.27G/1.27G [00:21<00:00, 59.5MB/s]


In [ ]:
!unzip /content/car_img-validation.zip

Archive:  /content/car_img-validation.zip
   creating: validation/
  inflating: validation/day_00004.jpg  
  inflating: validation/day_00004.xml  
  inflating: validation/day_00011.jpg  
  inflating: validation/day_00011.xml  
  inflating: validation/day_00018.jpg  
  inflating: validation/day_00018.xml  
  inflating: validation/day_00024.jpg  
  inflating: validation/day_00024.xml  
  inflating: validation/day_00033.jpg  
  inflating: validation/day_00033.xml  
  inflating: validation/day_00036.jpg  
  inflating: validation/day_00036.xml  
  inflating: validation/day_00041.jpg  
  inflating: validation/day_00041.xml  
  inflating: validation/day_00043.jpg  
  inflating: validation/day_00043.xml  
  inflating: validation/day_00045.jpg  
  inflating: validation/day_00045.xml  
  inflating: validation/day_00047.jpg  
  inflating: validation/day_00047.xml  
  inflating: validation/day_00049.jpg  
  inflating: validation/day_00049.xml  
  inflating: validation/day_00053.jpg  
  inflating: 

In [ ]:
import os
import xml.etree.ElementTree as ET

# Define paths
input_folder = "/content/validation"      # Folder containing XML files
images_folder = "/content/validation"     # Folder containing JPG files
labels_folder = "content/dataset/labels"  # Folder to save YOLO format .txt files

# Ensure the labels folder exists
os.makedirs(labels_folder, exist_ok=True)

# Function to get image size from JPG file (with error handling)
def get_image_size(image_path):
    try:
        with Image.open(image_path) as img:
            return img.size  # Returns (width, height)
    except FileNotFoundError:
        print(f"Warning: Image file not found: {image_path}")
        return None

# Function to convert XML to YOLO format
def xml_to_yolo(xml_file, images_folder, labels_folder):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
    except ET.ParseError:
        print(f"Error: Invalid XML file: {xml_file}")
        return

    # Get the corresponding JPG file name based on the XML filename
    xml_filename = os.path.splitext(os.path.basename(xml_file))[0]  # Extract filename without .xml
    jpg_file = xml_filename + ".jpg"  # Construct the image file name
    jpg_path = os.path.join(images_folder, jpg_file)

    # Skip if JPG file does not exist
    if not os.path.exists(jpg_path):
        print(f"Warning: Skipping {xml_file} (JPG file {jpg_file} not found)")
        return

    # Get image size from the JPG file
    image_size = get_image_size(jpg_path)
    if image_size is None:
        return  # Skip if image size couldn't be retrieved

    img_width, img_height = image_size

    # Create .txt file name based on the XML filename
    txt_filename = xml_filename + ".txt"
    txt_file = os.path.join(labels_folder, txt_filename)

    with open(txt_file, "w") as f:
        for obj in root.findall("object"):
            name = obj.find("name").text
            if name == "کل ناحیه پلاک":  # Only process "کل ناحیه پلاک"
                bbox = obj.find("bndbox")
                try:
                    xmin = float(bbox.find("xmin").text)
                    ymin = float(bbox.find("ymin").text)
                    xmax = float(bbox.find("xmax").text)
                    ymax = float(bbox.find("ymax").text)
                except ValueError:
                    print(f"Error: Invalid bbox values in {xml_file}")
                    continue

                # Convert to YOLO format
                x_center = (xmin + xmax) / 2 / img_width
                y_center = (ymin + ymax) / 2 / img_height
                width = (xmax - xmin) / img_width
                height = (ymax - ymin) / img_height

                # Write to .txt file (class_id is 0 for "کل ناحیه پلاک")
                f.write(f"0 {x_center} {y_center} {width} {height}\n")

# Process all XML files
for xml_file in os.listdir(input_folder):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join(input_folder, xml_file)
        xml_to_yolo(xml_path, images_folder, labels_folder)

print("Conversion complete!")


Conversion complete!


In [ ]:

images_and_xmls_dir = '/content/validation'
labels_dir = '/content/content/dataset/labels'

dataset_dir = '/content/dataset'
os.makedirs(os.path.join(dataset_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'labels'), exist_ok=True)

for file_name in os.listdir(images_and_xmls_dir):
    if file_name.endswith('.jpg'):
        src_image = os.path.join(images_and_xmls_dir, file_name)
        dst_image = os.path.join(dataset_dir, 'images', file_name)
        shutil.copy(src_image, dst_image)

for file_name in os.listdir(labels_dir):
    if file_name.endswith('.txt'):
        src_label = os.path.join(labels_dir, file_name)
        dst_label = os.path.join(dataset_dir, 'labels', file_name)
        shutil.copy(src_label, dst_label)

In [ ]:

labels_dir = '/content/dataset/'

drive_labels_dir = '/content/drive/MyDrive/val_dataset/'

shutil.copytree(labels_dir, drive_labels_dir)

'/content/drive/MyDrive/val_dataset/'

In [ ]:
!yolo predict model='/content/drive/MyDrive/content/runs/detect/train3/weights/best.pt' source='/content/drive/MyDrive/val_dataset/images' plots=True

Ultralytics 8.3.80 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs

image 1/2120 /content/drive/MyDrive/val_dataset/images/day_00004.jpg: 640x480 1 license_plate, 73.6ms
image 2/2120 /content/drive/MyDrive/val_dataset/images/day_00011.jpg: 640x480 1 license_plate, 12.4ms
image 3/2120 /content/drive/MyDrive/val_dataset/images/day_00018.jpg: 640x480 1 license_plate, 12.4ms
image 4/2120 /content/drive/MyDrive/val_dataset/images/day_00024.jpg: 640x480 1 license_plate, 12.4ms
image 5/2120 /content/drive/MyDrive/val_dataset/images/day_00033.jpg: 480x640 2 license_plates, 45.3ms
image 6/2120 /content/drive/MyDrive/val_dataset/images/day_00036.jpg: 480x640 1 license_plate, 12.5ms
image 7/2120 /content/drive/MyDrive/val_dataset/images/day_00041.jpg: 480x640 1 license_plate, 12.5ms
image 8/2120 /content/drive/MyDrive/val_dataset/images/day_00043.jpg: 640x640 1 license_plate, 16.3ms
image 9/2120 /co

In [ ]:

labels_dir = '/content/runs'

drive_labels_dir = '/content/drive/MyDrive/val_runs'

shutil.copytree(labels_dir, drive_labels_dir)

'/content/drive/MyDrive/val_runs'